In [ ]:
import numpy as np
import scipy
import matplotlib.pylab as plt

Linear Advection
==

Today we will start to numerically solve equations that kinda look like the Euler equations. Next time we will solve Euler's equations.  The difficulty, as we have seen in class, is that Euler's equations are non-linear.  This means, for example, in order to solve for the evolution of density we also need to solve for the evolution of the velocity (which also means we need to evolve the pressure, or energy).  

We will start slowly, looking at just one equation that is linear.  The so-called linear advection equations.  You can think of this equation as a simplified continuity equation in one dimension with a constant velocity, $u$.

$$\partial_t \rho + u \partial_x \rho = 0$$

Since $u$ is constant, this equation is now linear!  Given initial data, $\rho^0(x)$, the general solution for the time dependent density is, $\rho(x,t) = \rho^0(x-ut)$.  However, we will try and do this numerically.  It forms a great jumping off point for numerically solving the Euler equations, which do bring their own interesting aspects to the table.

Intial Conditions:
==

In [ ]:
#define some initial data
def initial_data_gausspulse(xL=0,xR=1,amplitude=1,npoints=64):
    dx = (xR-xL)/npoints
    xmid = (xR+xL)/2.0
    x = np.linspace(xL,xR,npoints,endpoint=False)+dx/2.0
    y = amplitude*scipy.signal.gausspulse(x-xmid,fc=6,retenv=True)[1]
    return x,y

In [ ]:
#generate and plot initial data, include labels, legends
npoints=128 #work with multiples of 2 to make the error estimate more easy

plt.plot(,label="Intial Data")
plt.ylabel()
plt.xlabel()
plt.legend()
plt.show()

Solving the Evolution Numerically
==

Let's make our lives easy here.  We can set the speed to 1 everywhere (in the linear advection equation the speed is constant.  We will make our boundary conditions periodic, so that when the wave propagates off the right it will return on the left.  We will evolve the equation of a number of time steps $n_\mathrm{step}$ with a $\Delta t$ of $1/n_\mathrm{step}$ so that after $n_\mathrm{step}$ the time past is 1 unit and the wave should return back to its original location, and we can see how well we did.

First we need to discretize our equation, and we have many options.  Not all will be good, and some will be bad. There are two things we need to discretize, the first is the time derivative and the second in the spatial derivative. We often call the quantity in the time derivated the "state variable" (here there is only $\rho$, but there may be more if you have several equations to solve).  The quantity in the spatial derivative is the "flux" since it represents the flux (state variable * velocity; here we have taken $u$ outside the derivative, but that was because it is constant, in the Euler equations it shows up inside the derivative).  A first guess at this discretization might be the following, called FTCS.  This stands for Forward Time, Centered Space, it transforms the continuous differential equation,

$$\partial_t \rho + u \partial_x \rho = 0\,,$$

into the following discretization,

$$\frac{\rho_i^{n+1} - \rho_i^{n}}{\Delta t}  + u \frac{\rho_{i+1} - \rho_{i-1}}{2\Delta x} = 0\,.$$

The $\rho_i^{n}$ notation here stands for the density in the $i$th zone (we have `npoints` of them in our example above) at the time level $n$ (we plan to evolve for $n_\mathrm{step}$ levels).  Our initial data above is $\rho_i^0$.  Here we are using "cell centered" quantities, that means that $\rho^n_i$ is the density at the $n$th time level in the center of the $i$th zone.  Notice above in the initial conditions we defined that $x$ positions to be in the center of the cell.

Solving for $\rho_i^{n+1}$ gives,

$$\rho_i^{n+1}  =  \rho_i^{n} + \Delta t [u \frac{\rho^n_{i+1} - \rho^n_{i-1}}{2\Delta x}]\,.$$

Note, this may start to look familiar from last lab. We are applying Euler's method (i.e. a one step, first order Runge-Kutta algorithm) to evolve the density forward in time.  Unlike last lab, we are now doing it for `npoints`, so we really have `npoints` differential equations that we are solving. Another added complication is the differential equations are semicoupled, since the evolution of the $i$th density depends on the $i-1$ and $i+1$ density. Luckily, this method uses only densities at the $n$ time level, and not the $n+1$ time level, then we would need to solve the equations simutaneously, as it is here, we just need to know the full $n$ state everywhere to find the $n+1$ state locally.

To get higher order time evolution, we can use higher order Runge-Kutta methods, like the midpoint from last time.  Solving the hydro equations (or the linear advection equation) this way is called the "Method of lines".  There are other methods, the so called characterized tracing method, which folds in the evolution of the quantity into the spatial derivative terms, we will not do that here.


In [ ]:
#create a function to evolve the density field one time step.
#It takes in the coordinates (x) initial density field (rhon),
#time step (\Delta t), and advection speed (u) and uses the
#FTCS method to determine the next density

#I've done this one for you, you can base the rest of of this
def evolve_one_FTCS(x,rhon,Deltat,speed):
    npoints=len(rhon)
    rhonp1 = np.zeros(npoints)
    Deltax = x[1]-x[0]

    #loop over zones
    for i in range(npoints):
        #determine the spatial flux term, for FTCS, that is  u \frac{\rho^n_{i+1} - \rho^n_{i-1}}{2\Delta x}]
        #mind the boundary conditions
        if i==npoints-1:
            CSflux = (rhon[0]-rhon[i-1])/(2.0*Deltax) #use rho[0] instead of rho[i+1] since that doesn't exist
        else:
            CSflux = (rhon[i+1]-rhon[i-1])/(2.0*Deltax)

        rhonp1[i] = rhon[i] - speed*Deltat*CSflux
    
    return rhonp1

In [ ]:
#evolve the equation for one full period.  We often take a time step that is a
#fraction of crossing time of the computational zone, the fraction is called
#the Courant number, C. It generaly must be less than some maximum that depends
#on your evolution equation and method as well as the number of dimensions.
#So in this case, the crossing time is Deltax/speed. Try taking C=0.25.  You can
#try different C's, but make sure 1/C is an integer so you can get exactly an
#integer number of periods
C=0.25
speed=1
npoints=128
x,rho0 = initial_data_gausspulse(xL=0,xR=1,amplitude=2,npoints=npoints)
nsteps = int(npoints/C+0.001) #this should be a number of steps to ensure 1 period 
Deltax = x[1]-x[0]
rhon=rho0
for i in range(nsteps):
    #set time step
    Deltat = 
    
    #evolve the density
    rhonp1 = 
    
    #set n state to the n+1 state and go to the next time step
    rhon=rhonp1
    
rhoend = rhon

#plot rho0 (Exact) and rhoend (evolved 1 period)

Hmmmm...
==

That didn't really work, try changing C to a smaller number, try increasing the resolution (i.e. increase `npoints`).  Does anything work?  Try plotting just a few steps, when do things go wrong?

This method, FTCS, is unstable and not a good approach to solving numerical equations.  This isn't really a course in stabilty of numerical methods, so we'll just move on, but you can see that numerically methods do not always work even if you crank up the resolution, so care must be taken to ensure the equations make sense.

Upwinding
==

One problem with the FTCS is if the fluid is traveling in a particular direction, say from zone $i$ to zone $i+1$, the actual content of the flux through the cell boundary never has the $i+1$ stuff in it.  For this reason, common schemes for solving advection equations involve "upwinding" this uses the direction of the flow to make informed choices about the value of the flux. 

We can start to think of the flux term as the following:

$$u\partial_x \rho = u \frac{\rho_{i+1/2} - \rho_{i-1/2}}{\Delta x}$$

where our discretization of the flux term is kinda accounting for the stuff leaving through the upper edge of the cell, the $i+1/2$ interface marking the boundary between the $i$ and $i+1$ cells, and entering through the bottom edge, the $i-1/2$ interface marking the boundary between the $i-1$ and $i$ cells. This concept will become more important when we go to solve the Euler equations and become concerned with conservation of things like mass, momentum, and energy.  In the case of upwinding, we take the flux at the interface $i+1/2$ to be either the flux evaluated at the cell center if cell $i$ or $i+1$, depending on the velocity.  This is one of the simplist examples of a Riemann problem. The Riemann problem aims to determine the flux at the interface ($F_{i+1/2}$) through a potentially complicated function $\mathcal{R}$ of the left and right fluxes (or more generally the left and right state variables). We define a Riemann problem for upwinding the advection equation as

\begin{equation}
\rho_{i+1/2} = \mathcal{R}(\rho_{i},\rho_{i+1})=
 \begin{cases}
    \rho_{i},& \text{if } u\geq 0\\
    \rho_{i+1},& \text{if } u < 0
\end{cases}\,,
\end{equation}
and

\begin{equation}
\rho_{i-1/2} = \mathcal{R}(\rho_{i-1},\rho_{i})=
 \begin{cases}
    \rho_{i-1},& \text{if } u\geq 0\\
    \rho_{i},& \text{if } u < 0
\end{cases}\,.
\end{equation}

In [ ]:
#create a function to evolve the density field one time step.
#It takes in the coordinates (x) initial density field (rhon),
#time step (\Delta t), and advection speed (u) and uses the
#upwind method to determine the next density
def evolve_one_upwind(x,rhon,Deltat,speed):
    npoints=len(rhon)
    rhonp1 = np.zeros(npoints)
    Deltax = x[1]-x[0]
    #loop over zones
    for i in range(npoints):
        #determine the spatial flux term, for upwind, that is  u \frac{\rho^n_{i} - \rho^n_{i-1}}{\Delta x}]
        #if u>0 and u \frac{\rho^n_{i+1} - \rho^n_{i}}{\Delta x}] if u<0
        #mind the boundary conditions
       
        ...
    
        rhonp1[i] = rhon[i] - Deltat*speed*UWflux
    
    return rhonp1

In [ ]:
C=0.25
npoints=128
speed=-1
x,rho0 = initial_data_gausspulse(xL=0,xR=1,amplitude=2,npoints=npoints)
nsteps = int(npoints/C+0.001)
Deltax = x[1]-x[0]
rhon=rho0

for i in range(nsteps):
    #set time step
    Deltat = 
    
    #evolve the density
    rhonp1 = 
    
    #set n state to the n+1 state and go to the next time step
    rhon=rhonp1
    

rhoend = rhon

#plot rho0 (Exact) and rhoend (evolved 1 period)

Quantifying Errors
==

How well is our upwinding method doing?  We can quantify the error and then see how it changes with resolution or time step size.  This way we can determine, like last lab, the order or our method. As a note, since we discretized in two dimensions (time and space) we, in principle, have an order for each method.  This can make it difficult to fully ascertain the order of particular methods.

A very common error measure in numerical hydrodynamics is the $L2$ norm.

$$l^2 = \sqrt{\frac{1}{N} \sum_{i=0}^N (\rho_i-\rho_\mathrm{exact})^2}$$

Determine the $L2$ norm for the upwinding method for several values of `npoints` to see the converegence there. Is this first or second order?

In [ ]:
for npoints in [32,64,128,256,512,1024]:
    
    #repeat the 1 period evolution here for each npoints in the list
    
    #then calculate the L2norm between the endstate and the initial (exact) state
    L2norm = ...
    print(L2norm," for npoints=",npoints)

So is this what you expect, recall that if the $\Delta x$ goes down by a factor of 2, then a first-order scheme should see the error go down by a factor of 2.  If the scheme was $n$ order scheme, it should go down by a factor of $2^n$

Increasing the order
==

One way to increase the order is to do a better job at getting the input to the Riemann problem.  Above we determined the interface value from using the cell centered values.  We show the consequence of this below, where we show the 'exact' function as a line and our assumption of what the field looks like when we have a resolution of only 32 points.  

In [ ]:
npoints=32
x,rho0 = initial_data_gausspulse(xL=0,xR=1,amplitude=2,npoints=npoints)
plt.bar(x, rho0,width=1.0/npoints)
x,rho0 = initial_data_gausspulse(xL=0,xR=1,amplitude=2,npoints=1024)
plt.plot(x, rho0,'k')
plt.xlabel('x')
plt.ylabel(r'$\rho^0$')
plt.show()

In princple we can "reconstruct" the state variables on the interface as use these in the Riemann problem instead. Then,

\begin{equation}
\rho_{i+1/2} = \mathcal{R}(\rho^L_{i+1/2},\rho^R_{i+1/2})=
 \begin{cases}
    \rho^L_{i+1/2},& \text{if } u\geq 0\\
    \rho^R_{i+1/2},& \text{if } u < 0
\end{cases}\,,
\end{equation}

We have kinda done this already, just our reconstruction was bad.  It is what is called piecewise-constant reconstruction. Another common method of reconstruction is piecewise-linear reconstruction. However, when the reconstruction gets fancy we often have problems.  One problem with piecewise-linear is that the slope can cause us to overshoot the value of the state variables in either zone.  Then we need to limit it.  

The piecewise-linear, total variation diminishing method fixes this. Given a set of cell centered values $..., \rho_{i-1}, \rho_i, \rho_{i+1}, ...$ on an evenly spaced grid $\Delta x$ apart, the TVD method provides a slope of the state variable within the cell so we can define the $x$ dependence inside the zone: $\rho(x) = \rho_i + (x-x_i)S_i$. Here $S_i$ is the slope of the state variable within the zone.  With this definition, we can evaluate the state variable on the interface

$$\rho^L_{i+1/2} = \rho_{i} + \frac{\Delta x}{2} \mathrm{limiter}\left(\frac{\rho_i-\rho_{i-1}}{\Delta x}, \frac{\rho_{i+1}-\rho_i}{\Delta x}\right)$$
$$\rho^R_{i-1/2} = \rho_{i} - \frac{\Delta x}{2} \mathrm{limiter}\left(\frac{\rho_i-\rho_{i-1}}{\Delta x}, \frac{\rho_{i+1}-\rho_i}{\Delta x}\right)$$

Here we have replaced the slope with a function called `limiter`, for which we have lots of choices, a common one is the `minmod` limiter

\begin{equation}
\mathrm{minmod}(a,b) =  \begin{cases}
    a,& \text{if } ab>0\ \&\ \mathrm{abs}(a) <= \mathrm{abs}(b)\\
    b,& \text{if } ab>0\ \&\ \mathrm{abs}(b) < \mathrm{abs}(a)\\
    0,& \text{if } ab<0
\end{cases}\,.
\end{equation}

essentially, this function takes as arguements the differences in the state variables from surrounding zones and if the slope of the state variable changes then we revert to piecewise constant (i.e. $S_i=0$).  Otherwise, we take the smallest (absolute value) slope as the estimate for the slope of the state variable in the zone. 

In all cases, the integral over the cell is the same (just $\rho_i \times \Delta x$). This is because we are defining the density in the cell to be of the form $\rho(x) = \rho_i + (x-x_i)S_i$.  When this is integrated across the zone slope part cancels out (i.e. $\int_{x_i-\Delta x/2}^{x_i+\Delta x/2} (x-x_i) S_i dx = 0$). 

Implement this TVD scheme and check the order, I have built it a bit for you, and have created guard cells to make the treatment of the boundary conditions easier to handle.

In [ ]:
#create a function to evolve the density field one time step.
#It takes in the coordinates (x) initial density field (rhon),
#time step (\Delta t), and advection speed (u) and uses the
#upwind method to determine the next density, However, we use
#the piecewise-linear TVD method to determine the interface values first
def minmod(a,b):
    if a*b>0:
        if np.abs(a)<np.abs(b):
            return a
        else:
            return b
    else:
        return 0

def evolve_one_upwind_tvd(x,rhon,Deltat,speed):
    npoints = len(rhon)
    rhonp1 = np.zeros(npoints)
    Deltax = x[1]-x[0]
        
    #guard cells to make equations simpler, periodic boundaries
    #now you don't have to worry about the ends on the array since we add some buffers in.
    #Now, with a second-order method, we need 2 buffer zones, not just 1 like above.
    ng=2
    data = np.zeros(npoints+2*ng)
    datanp1 = np.zeros(npoints+2*ng)
    data[2:-2] = rhon #this is the actual data we will evolve, following are just copies for the periodic boundaries
    data[0] = rhon[-2]
    data[1] = rhon[-1]
    data[-2] = rhon[0]
    data[-1] = rhon[1]
    
    #loop over real zones, not the guard zones
    for i in range(ng,npoints+ng):
        #determine the spatial flux term, for upwind, that is  u \frac{\rho^n_{i} - \rho^n_{i-1}}{\Delta x}]
        #if u>0 and u \frac{\rho^n_{i+1} - \rho^n_{i}}{\Delta x}] if u<0
        if speed > 0:
            #need the rho^L_{i+1/2} state for the i+1/2 interface, call it the Fp (p for plus) and
            #the rho^L_{i-1/2} state for the i-1/2 interface, call it the Fm (m for minus)
            Fp = ...
            Fm = ...
        else:
            #need the rho^R_{i+1/2} state for the i+1/2 interface, call it the Fp (p for plus) and
            #the rho^R_{i-1/2} state for the i-1/2 interface, call it the Fm (m for minus)
            Fp = ...
            Fm = ...
            
        TVDflux = ...
            
        datanp1[i] = data[i] - Deltat*speed*TVDflux
    
    rhonp1 = datanp1[ng:-ng] #just get out the real data and return
    return rhonp1

In [ ]:
C=0.25
speed=-1
npoints=256
x,rho0 = initial_data_gausspulse(xL=0,xR=1,amplitude=2,npoints=npoints)
nsteps = int(npoints/C+0.001)
Deltax = x[1]-x[0]

#lets do a TVD evolution and a piecewise constant evolution and compare
rhon=rho0
for i in range(nsteps):
    #set time step
    Deltat = 
    
    #evolve the density
    rhonp1 = 
    
    #set n state to the n+1 state and go to the next time step
    rhon=rhonp1

rhoend_tvd = rhon

#reset n state to initial data
rhon=rho0
for i in range(nsteps):
    #set time step
    Deltat = 
    
    #evolve the density
    rhonp1 = 
    
    #set n state to the n+1 state and go to the next time step
    rhon=rhonp1


    rhoend_pc = rhon


plt.plot(x,rhoend_tvd,label="TVD")
plt.plot(x,rhoend_pc,label="PC")
plt.plot(x,rho0,label="Exact")
plt.legend()
plt.xlabel('x')
plt.ylabel(r'$\rho^0$')
plt.show()

So TVD does better at maintaining the peak of the pulse as it advects, but it tends to spread it out too.

In [ ]:
#Now look at the errors to test the convergence 
for npoints in [32,64,128,256,512]:

    #evolve based on npoints
    
    L2norm = ...
    print(L2norm," for npoints=",npoints)
    plt.plot(,,label=str(npoints)) #final profile for
    
plt.plot(x,rho0,label="Exact")

plt.legend()
plt.show()

What gives...
==

Even though we have a second order scheme for the spatial flux calculation, we still only have first order convergence.  This is because our time evolution is still only first order. The error we are seeing in dominated by the time stepping, not the spatial derivatives. Lets drop C until we are instead dominated by spatial error, then see if we have 2nd order spatial convergence

In [ ]:
#Now look at the errors to test the convergence in time
for C in [0.5,0.25,0.125,0.0625,0.03125,0.015625]:
    npoints=256

    #evolve
    
    L2norm = ...
    print(L2norm," for C=",C)
    plt.plot(,,label=str(C))

plt.plot(x,rho0,label="Exact")

plt.legend()
plt.show()

In [ ]:
#OK, so if C=0.015625 the error seems to be dominated by the spatial
#resolution (since dropping C didn't reduce the error)
for npoints in [32,64,128,256]:
    C=0.015625
    
    #evolve
    
    L2norm = ...
    print(L2norm," for npoints=",npoints)
    plt.plot(x,rhoend_tvd,label=str(npoints))

plt.plot(x,rho0,label="Exact")

plt.legend()
plt.show()

OK, so better than first order, the TVD scheme, when it limits the slopes formally is no longer second order, so some deviation from the perfect expectation is ok.

Full second order
==
Implement a two stage Runge-Kutta that is second order accurate (like the midpoint method from last time) and see if you can acheive second order convergence for reasonable value of C.

In [ ]:
def evolve_one_upwind_tvd_rk2(x,rhon,Deltat,speed):
    npoints = len(rhon)
    rhonp1 = np.zeros(npoints)
    Deltax = x[1]-x[0]
        
    #guard cells to make equations simpler, periodic boundaries
    ng=2
    data = np.zeros(npoints+2*ng)
    datan = np.zeros(npoints+2*ng)
    datanphalf = np.zeros(npoints+2*ng)
    datanp1 = np.zeros(npoints+2*ng)
    data[2:-2] = rhon[:]
    data[0] = rhon[-2]
    data[1] = rhon[-1]
    data[-2] = rhon[0]
    data[-1] = rhon[1]
    datan = data*1.0
    
    #first stage
    
    
    rhonphalf = datanphalf[ng:-ng]
    
    
    #second stage
    #reset boundaries as variables have change
    data[2:-2] = rhonphalf[:]
    data[0] = rhonphalf[-2]
    data[1] = rhonphalf[-1]
    data[-2] = rhonphalf[0]
    data[-1] = rhonphalf[1]
    
    rk_dt = ...
    
    #evolve
    
    rhonp1 = datanp1[ng:-ng]
    return rhonp1

In [ ]:
#show a higher order convergence with Npoints for a reasonable C